In [1]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_42027/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [2]:
llm.invoke('hi')

'Hello! How can I assist you today? If you have any questions or need information, feel free to ask.'

In [3]:
llm.invoke('سلام. خوبی؟')

'سلام! بله، خیلی خوبم، صبر کنید چطوری می\u200cشود؟ شما چطور هستید؟'

In [4]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [5]:
llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [8]:
# llm_math.run("what is 13 raised to the .3432 power?")

In [6]:
i = 0
example_query = "What is 551368 divided by 82"
llm_math.invoke(example_query)



> Entering new LLMMathChain chain...
What is 551368 divided by 82To solve the problem of dividing 551368 by 82 using Python's `numexpr` library, we first need to translate the mathematical expression into a format that can be executed by `numexpr.evaluate`. Here’s how you can do it:

```text
551368 / 82
```

Now, let's evaluate this expression using `numexpr`:

...numexpr.evaluate("551368 / 82")...

The output of running this code will give us the result of the division.

Answer: 6722.0

So, 551368 divided by 82 is 6722.0.
> Finished chain.


{'question': 'What is 551368 divided by 82',
 'answer': 'Answer:  6722.0\n\nSo, 551368 divided by 82 is 6722.0.'}

## understand which formula is selected

In [15]:
from langchain.chains import LLMChain
from langchain.chains.router import LLMRouterChain
from langchain.prompts import PromptTemplate

In [16]:
# Define prompt templates for each formula chain
house_price_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question asks about a house price. Apply Formula1: {question}"
)
other_task_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question seems to be about another task. Apply Formula2: {question}"
)


In [ ]:
# Create LLM chains for each formula
house_price_chain = LLMChain(llm=llm, prompt=house_price_prompt)
other_task_chain = LLMChain(llm=llm, prompt=other_task_prompt)

In [ ]:
# Create a router chain that routes based on the question content
router_chain = LLMRouterChain.from_llm_and_prompts(
    llm=llm,  
    destination_chains={
        "house_price": house_price_chain,
        "other_task": other_task_chain,
    },
    # default_chain=default_chain  # if applicable
    
)



AttributeError: from_llm_and_prompts

In [19]:
# Use the router chain on your input
input_text = "What is the price of the house?"
result = router_chain.run(input_text)
print(result)


NameError: name 'router_chain' is not defined